In [1]:
import pandas as analytics
import numpy as maths
import time
from locations import rated_path
import warnings
warnings.filterwarnings("ignore")

In [2]:
def feedback(user_feeback,feedback_values,t):                      # instead of actual fut definition, its changed a bit, i.e. if in the selected list there is a movie user likes then it is feedback is 1 else 0
    value = 0
    try :
        if 0 <= int(user_feedback) <= 2 :
            value = 1
        elif int(user_feedback) == -1 :
            value = 0
        else : 
            
            value = -1
    except ValueError as e :
            value = -1
    feedback_values.insert(t,value)
    return feedback_values
    
def generate(state,old_movies): 
    movie_ids = []
    for _class in state.index : 
        df_temp = df_rated_movies[(df_rated_movies['mask'] == 1)][df_rated_movies['class'] == _class].sample(n = int(state[_class])+1)
        movie_ids = movie_ids + df_temp['movie_id'].to_list()
        
    if len(movie_ids) < N : 
        diff = N - len(movie_ids)
        additional_ids = df_rated_movies[df_rated_movies['mask'] == 1][~df_rated_movies['movie_id'].isin(movie_ids)].sample(n = diff)['movie_id'].tolist()
        movie_ids = movie_ids + additional_ids
    df_candidates = df_rated_movies[df_rated_movies['movie_id'].isin(movie_ids)]
   
    if maths.random.random() > 0.08 :                        # exploitation
        df_candidates = df_candidates.iloc[:N]
    else : df_candidates = df_candidates.sample(n = N)      # exploration
    df_candidates = df_candidates.drop('mask',axis = 1) 
    df_interested[df_interested['movie_id'].isin(movie_ids)]['mask'] = 0
    df_rated_movies[df_rated_movies['movie_id'].isin(old_movies)]['mask'] = 1
    return df_candidates,movie_ids
    

def find_rewards(values,rewards, feedback_value):
    gamma = 0.2
    if feedback_value > 0 : value = 1 
    else : value = -0.2  
    values.append(value*gamma**t)
    rewards.append(sum(values[:t]))
    return values, rewards
    

def transition(P,old_state):                              #equivalent to the RNN function. So it is the most complex and challenging function
    alpha = 0.6
    next_state = (P[P['rating'] > 0]['class'].value_counts()*alpha + old_state * (1-alpha)).fillna(0)
    next_state = next_state / next_state.sum()
    next_state = round(next_state * N)
    if next_state.sum() < N :
        next_state = old_state

    return next_state

def restart():
    t = 0 
    df_rated_movies['mask'] = 1
    rewards = []
    values = []
    feedback_values = [0]
    old_movies = []
    states = []
    
    initial_state = df_interested['class'].value_counts(normalize = True)*N   # State Space ( discrete :-) ) [Out of 3 items , how many belong to each class is each state. That is 4 non-negative integers add upto 3]. 20 ways are there.
    P , old_movies = generate(initial_state, old_movies)   # Action Space ( discrete :-) )
    states.append(initial_state)
    
    values , rewards = find_rewards(values , rewards, feedback_values[t])
    return rewards, values, feedback_values, old_movies, states, initial_state, P

In [3]:
df_rated_movies = analytics.read_csv(rated_path)
user_id = df_rated_movies['user_id'].unique()[0]
df_rated_movies = df_rated_movies.drop(['user_id','timestamp'],axis = 1)
df_movies_mapping = df_rated_movies[['movie_id','movie_title']]
df_rated_movies

,movie_id,movie_title,rating,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),1,0,0,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),1,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,7,Twelve Monkeys (1995),1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
535,1480,Herbie Rides Again (1974),0,0,1,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
536,1490,Fausto (1993),0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
537,1518,Losing Isaiah (1995),1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
538,1521,Mr. Wonderful (1993),0,0,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0


In [4]:
genres = list(df_rated_movies.drop(['movie_id','movie_title','rating'],axis = 1).columns)
selected_genres = df_rated_movies[genres].sum().sort_values(ascending = False).reset_index(drop = False).iloc[:2]['index'].tolist()
print("Most Popular user #",user_id," are ",selected_genres,"\n")

df_rated_movies['class'] = df_rated_movies[selected_genres[0]] * 2 + df_rated_movies[selected_genres[1]]
class_mapping = df_rated_movies[selected_genres + ['class']].drop_duplicates().sort_values('class').set_index('class')    # mapping of drama ,comedy movie to respective class
classes = class_mapping.index.tolist()

selected_columns = ['movie_id','rating','class']
df_rated_movies = df_rated_movies[selected_columns]
df_rated_movies

Most Popular user # 450  are  ['Drama', 'Comedy'] 



,movie_id,rating,class
0,1,1,1
1,2,1,0
2,3,1,0
3,4,0,3
4,7,1,2
...,...,...,...
535,1480,0,1
536,1490,0,1
537,1518,1,2
538,1521,0,1


In [5]:
N = 3

In [6]:
df_interested = df_rated_movies.copy()
df_interested['F'] = df_interested['rating'].apply(lambda x : 1 if x > 0 else 0)
df_interested = df_interested[df_interested['F'] > 0].drop('F',axis=1)
df_interested

,movie_id,rating,class
0,1,1,1
1,2,1,0
2,3,1,0
4,7,1,2
5,10,1,2
...,...,...,...
529,1425,1,3
530,1435,1,1
532,1444,1,1
533,1446,1,1


In [7]:
df_raw = analytics.DataFrame(data = [maths.arange(0,N+1)]).T
df_merge = analytics.merge(df_raw, df_raw, how = 'cross',suffixes=('_1','_2'))
df_merge = analytics.merge(df_merge, df_raw, how = 'cross',suffixes=('_x','_y'))
df_merge = analytics.merge(df_merge, df_raw, how = 'cross')
df_merge['sum'] = df_merge.sum(axis = 1)
df_all_states = df_merge[df_merge['sum'] == N].drop('sum',axis = 1).reset_index(drop = True)
df_all_states.columns = list(maths.arange(N+1))
all_states = []
for i in range(len(df_all_states)) :
    all_states.append(df_all_states.iloc[i].astype('float64'))

In [8]:
t = 0
rewards, values, feedback_values, old_movies, states, initial_state, P = restart()
recommendations = []

# print("Initial State")
# print("==============")
# print(initial_state)

print("Initial Rewards :",rewards[-1])

while t <= len(df_interested) :
    t = t + 1
    state = transition(P,states[t-1])
    states.append(state)
    P , old_movies = generate(state,old_movies) 
    print("\n\n")
    print(P.merge(df_movies_mapping,on = 'movie_id')[['movie_title']].rename(columns={'movie_title':'Recommended Movies'}))
    print()
    user_feedback = input("[Instructions : \n If you liked any movie enter its index number, \n If you didn't like any movie enter -1. \n Any other input would end the loop] \n\n Did you like any movie ? \t")
    
    feedback_values = feedback(user_feedback,feedback_values,t)
    recommendations.append(P)
    values , rewards = find_rewards(values , rewards, feedback_values[t])
    print("Reward Earned :","{:.4f}".format(values[-1]))
    print("----------------------------------------------------")
    if feedback_values[t] == -1 :
        print("\n")
        print("-----------------------------------------------")
        print('| Wrong Value Entered ! Ending the simulation |')
        print("-----------------------------------------------")
        break    
    


states_policy = states
rewards_policy = rewards
recommendations_policy = recommendations

print("\n\n---------------------------------")
print("| Total Rewards Earned :","{:.4f}".format(rewards[-1]),"|")
print("---------------------------------")

Initial Rewards : 0



                     Recommended Movies
0                  Love Bug, The (1969)
1  Bridge on the River Kwai, The (1957)
2      Hunt for Red October, The (1990)



[Instructions : 
 If you liked any movie enter its index number, 
 If you didn't like any movie enter -1. 
 Any other input would end the loop] 

 Did you like any movie ? 	 0


Reward Earned : 0.2000
----------------------------------------------------



     Recommended Movies
0   Pulp Fiction (1994)
1        M*A*S*H (1970)
2  Event Horizon (1997)



[Instructions : 
 If you liked any movie enter its index number, 
 If you didn't like any movie enter -1. 
 Any other input would end the loop] 

 Did you like any movie ? 	 1


Reward Earned : 0.0400
----------------------------------------------------



           Recommended Movies
0      Aristocats, The (1970)
1  Blues Brothers, The (1980)
2    Full Metal Jacket (1987)



...

Reward Earned : 0.0080
----------------------------------------------------



       Recommended Movies
0    Seven (Se7en) (1995)
1  Aristocats, The (1970)
2          Top Gun (1986)



[Instructions : 
 If you liked any movie enter its index number, 
 If you didn't like any movie enter -1. 
 Any other input would end the loop] 

 Did you like any movie ? 	 -1


Reward Earned : -0.0003
----------------------------------------------------



        Recommended Movies
0        Get Shorty (1995)
1  Frighteners, The (1996)
2          Fantasia (1940)



[Instructions : 
 If you liked any movie enter its index number, 
 If you didn't like any movie enter -1. 
 Any other input would end the loop] 

 Did you like any movie ? 	 exit


Reward Earned : -0.0001
----------------------------------------------------


-----------------------------------------------
| Wrong Value Entered ! Ending the simulation |
-----------------------------------------------


---------------------------------
| Total Rewards Earned : 0.0477 |
---------------------------------
